In [1]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [2]:
using Revise
using LinearAlgebra
using Distributions
using PDMats
using StatsBase
using DataFrames, TimeSeries, XLSX
using PrettyTables

┌ Warning: attempting to remove probably stale pidfile
│   path = C:\Users\matsz\.julia\compiled\v1.11\DataFrames\AR9oZ_d1gGA.ji.pidfile
└ @ FileWatching.Pidfile C:\Users\matsz\.julia\juliaup\julia-1.11.2+0.x64.w64.mingw32\share\julia\stdlib\v1.11\FileWatching\src\pidfile.jl:249


In [4]:
include("../VARs/utils.jl")
includet("../bootstrap_model.jl")

In [5]:
df = DataFrame(XLSX.readtable("../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

freq = 12
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))


display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1473,1.8543,0.1012,0.4365,-2.9008,-0.6469,-0.1883,0.0258,2.1941,0.6874,0.1474
Price_10Y,0.0755,1.3969,0.0531,0.1909,-0.9866,0.3097,-0.0893,0.1022,1.9867,0.7037,0.0377


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1479,1.7607,0.0658,0.4053,-2.9112,-0.596,-0.2186,0.032,1.9119,0.445,0.1107
Price_10Y_CPI,0.078,1.279,0.0177,0.1544,-1.1677,0.2268,-0.1322,0.1437,2.0153,0.2268,0.0058


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0        0.0980477
 0.0980477  1.0

In [6]:
assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
data = to(from(real_returns[assets_names], start),end_d) 

scenarios = block_bootstrap(values(data), 10_000, block_size=60, random_block=true)

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  


,Total_Stock_CPI,Price_10Y_CPI
1,0.0635,0.0247
5,0.0632,0.0235
10,0.0619,0.0223
25,0.0623,0.0222


,Total_Stock_CPI,Price_10Y_CPI
1,0.1567,0.0865
5,0.0681,0.0415
10,0.0483,0.0311
25,0.0301,0.0204


,Total_Stock_CPI,Price_10Y_CPI
1,-0.7773,0.5244
5,-0.2652,0.2761
10,-0.2955,0.1245
25,-0.1677,0.0801


,Total_Stock_CPI,Price_10Y_CPI
1,1.1654,1.0231
5,-0.1454,0.6981
10,-0.2771,-0.0371
25,-0.0992,-0.1397


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.4252,-0.2905,-0.2157,-0.0182,0.0818,0.1658,0.286,0.3119,0.3653
5,-0.0959,-0.0737,-0.0514,0.0149,0.0725,0.1099,0.1696,0.1927,0.2104
10,-0.0514,-0.0362,-0.0229,0.0295,0.0656,0.0975,0.1346,0.1458,0.1604
25,-0.0101,0.0017,0.0118,0.0416,0.0634,0.0836,0.1098,0.1173,0.1277


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1572,-0.1238,-0.0996,-0.0341,0.0196,0.0785,0.1725,0.2263,0.285
5,-0.0737,-0.0578,-0.0414,-0.0035,0.0219,0.0496,0.0928,0.1146,0.1419
10,-0.0468,-0.037,-0.0271,0.0007,0.022,0.0427,0.0753,0.0865,0.0963
25,-0.0223,-0.0166,-0.0113,0.0083,0.0218,0.036,0.0564,0.0629,0.0703


In [7]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.406639  0.521378  0.539183  0.592069  0.720677

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 60.0  107.0  145.0  190.0  351.03

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.19893  0.376498  0.431628  0.496197  0.650372

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 79.0  156.0  223.0  335.0  658.0

In [51]:
rr = values(to(from(real_returns[assets_names], start),end_d) ) .- mean(values(to(from(real_returns[assets_names], start),end_d) ), dims=1)


scenarios = block_bootstrap(rr, 10_000, block_size=60, random_block=true)


scenarios[1,:,:] =  scenarios[1,:,:] .+ .046 ./ 12


scenarios[2,:,:] =  scenarios[2,:,:] .+ .02 ./ 12

835×10000 Matrix{Float64}:
  0.0288692     0.0221311    0.0171291    …   0.00827602    0.028719
  0.0279674    -0.0209067    0.0104963        0.0312384     0.0184496
 -0.0249681    -0.00237217  -0.000802425      0.0168531    -0.0393897
 -0.00480501    0.0119694    0.0147795        0.0270557    -0.045846
  0.000963775  -0.0178988    0.0285883        0.0293488    -0.0174949
 -0.0213192     0.00267608   0.0173859    …  -0.0304197    -0.0320622
  0.0206941     0.0200443    0.00789398      -0.0206414    -0.0131687
  0.00808764    0.0323329    0.0143397        0.000463922   0.0186337
  0.0159161     0.0377748   -0.018893         0.0134276    -0.0136375
 -0.00105215    0.0240808   -0.010601        -0.0430246    -0.0369565
  ⋮                                       ⋱                
 -0.00214197   -0.0459173   -0.00109844       0.0118662     0.0134764
  0.000306196  -0.00385186  -0.00865461       0.051438     -0.0148411
 -0.0121153    -0.0232654    0.0197403       -0.0403069    -0.0125516
 -0.0

In [52]:
periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)

print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  


,Total_Stock_CPI,Price_10Y_CPI
1,0.044,0.027
5,0.0427,0.0256
10,0.0428,0.0247
25,0.0425,0.0244


,Total_Stock_CPI,Price_10Y_CPI
1,0.155,0.086
5,0.0689,0.0417
10,0.0485,0.0312
25,0.0298,0.0201


,Total_Stock_CPI,Price_10Y_CPI
1,-0.8012,0.4974
5,-0.2807,0.3299
10,-0.3157,0.1602
25,-0.1668,0.1148


,Total_Stock_CPI,Price_10Y_CPI
1,1.2008,1.0675
5,-0.1773,0.8986
10,-0.2821,0.0283
25,-0.1335,-0.2006


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.4396,-0.314,-0.2353,-0.0405,0.0627,0.146,0.262,0.2916,0.3456
5,-0.1175,-0.0939,-0.0713,-0.007,0.052,0.0905,0.1501,0.1715,0.1883
10,-0.0722,-0.0572,-0.0439,0.0103,0.0473,0.0789,0.1152,0.1271,0.1403
25,-0.0293,-0.0182,-0.0082,0.0225,0.0435,0.0636,0.0894,0.0979,0.108


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1595,-0.1215,-0.0977,-0.0304,0.0232,0.079,0.1711,0.2286,0.2873
5,-0.0719,-0.056,-0.0393,-0.0006,0.0235,0.0507,0.0963,0.1154,0.1509
10,-0.0456,-0.0351,-0.0236,0.0035,0.024,0.0446,0.0791,0.09,0.1003
25,-0.0185,-0.0129,-0.0077,0.0101,0.0241,0.0382,0.0584,0.0649,0.0722


In [48]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.464108  0.548537  0.61719  0.675404  0.812231

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 77.97  158.0  213.0  302.0  579.0

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.193099  0.371434  0.413839  0.477163  0.632924

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 76.0  145.0  209.0  309.0  606.0